# Long Short-Term Memory (LSTM) Model
---

Lets first create a tiny LSTM network sample to understand the architecture of LSTM networks.

We need to import the necessary modules for our code. We need <b><code>numpy</code></b> and <b><code>tensorflow</code></b>, obviously. Additionally, we can import directly the <b><code>tensorflow.contrib.rnn</code></b> model, which includes the function for building RNNs.

In [1]:
import numpy as np
import tensorflow as tf
sess = tf.Session()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/conda/envs/python/lib/python3

We want to create a network that has only one LSTM cell. We have to pass 2 elements to LSTM, the <b>prv_output</b> and <b>prv_state</b>, so called, <b>h</b> and <b>c</b>. Therefore, we initialize a state vector, <b>state</b>.  Here, <b>state</b> is a tuple with 2 elements, each one is of size [1 x 4], one for passing prv_output to next time step, and another for passing the prv_state to next time stamp.

In [2]:
LSTM_CELL_SIZE = 4  # output size (dimension), which is same as hidden size in the cell

lstm_cell = tf.contrib.rnn.BasicLSTMCell(LSTM_CELL_SIZE, state_is_tuple=True)
state = (tf.zeros([1,LSTM_CELL_SIZE]),)*2
state

(<tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>,
 <tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>)

Let define a sample input. In this example, batch_size = 1, and  seq_len = 6:

In [3]:
sample_input = tf.constant([[3,2,2,2,2,2]],dtype=tf.float32)
print (sess.run(sample_input))

[[3. 2. 2. 2. 2. 2.]]


Now, we can pass the input to lstm_cell, and check the new state:

In [4]:
with tf.variable_scope("LSTM_sample1"):
    output, state_new = lstm_cell(sample_input, state)
sess.run(tf.global_variables_initializer())
print (sess.run(state_new))

LSTMStateTuple(c=array([[ 0.22646296,  0.45290354,  0.41990855, -0.39079046]],
      dtype=float32), h=array([[ 0.07295391,  0.32333648,  0.23629187, -0.12537599]],
      dtype=float32))


As we can see, the states has 2 parts, the new state c, and also the output h. Lets check the output again:

In [5]:
print (sess.run(output))

[[ 0.07295391  0.32333648  0.23629187 -0.12537599]]


<hr>
<a id="stacked_ltsm"></a>
<h2>Stacked LSTM</h2>
What about if we want to have a RNN with stacked LSTM? For example, a 2-layer LSTM. In this case, the output of the first layer will become the input of the second.

Lets start with a new session:

In [6]:
sess = tf.Session()

In [7]:
input_dim = 6

Lets create the stacked LSTM cell:

In [8]:
cells = []

Creating the first layer LTSM cell.

In [9]:
LSTM_CELL_SIZE_1 = 4 #4 hidden nodes
cell1 = tf.contrib.rnn.LSTMCell(LSTM_CELL_SIZE_1)
cells.append(cell1)

Creating the second layer LTSM cell.

In [10]:
LSTM_CELL_SIZE_2 = 5 #5 hidden nodes
cell2 = tf.contrib.rnn.LSTMCell(LSTM_CELL_SIZE_2)
cells.append(cell2)

To create a multi-layer LTSM we use the <b>tf.contrib.rnnMultiRNNCell</b> function, it takes in multiple single layer LTSM cells to create a multilayer stacked LTSM model.

In [11]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell(cells)

Now we can create the RNN from <b>stacked_lstm</b>:

In [12]:
# Batch size x time steps x features.
data = tf.placeholder(tf.float32, [None, None, input_dim])
output, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype=tf.float32)

Lets say the input sequence length is 3, and the dimensionality of the inputs is 6. The input should be a Tensor of shape: [batch_size, max_time, dimension], in our case it would be (2, 3, 6)

In [13]:
#Batch size x time steps x features.
sample_input = [[[1,2,3,4,3,2], [1,2,1,1,1,2],[1,2,2,2,2,2]],[[1,2,3,4,3,2],[3,2,2,1,1,2],[0,0,0,0,3,2]]]
sample_input

[[[1, 2, 3, 4, 3, 2], [1, 2, 1, 1, 1, 2], [1, 2, 2, 2, 2, 2]],
 [[1, 2, 3, 4, 3, 2], [3, 2, 2, 1, 1, 2], [0, 0, 0, 0, 3, 2]]]

we can now send our input to network, and check the output:

In [14]:
output

<tf.Tensor 'rnn/transpose_1:0' shape=(?, ?, 5) dtype=float32>

In [15]:
sess.run(tf.global_variables_initializer())
sess.run(output, feed_dict={data: sample_input})

array([[[ 0.02576734, -0.0189045 , -0.02119884, -0.04190461,
          0.01526202],
        [ 0.06672981, -0.04875976, -0.05820957, -0.12082575,
          0.03438251],
        [ 0.09472016, -0.078619  , -0.10110282, -0.18904437,
          0.0459863 ]],

       [[ 0.02576734, -0.0189045 , -0.02119884, -0.04190461,
          0.01526202],
        [ 0.08483623, -0.08444334, -0.07493887, -0.15172377,
          0.06263752],
        [ 0.12152544, -0.10320201, -0.1286984 , -0.25447777,
          0.06729159]]], dtype=float32)

As you see, the output is of shape (2, 3, 5), which corresponds to our 2 batches, 3 elements in our sequence, and the dimensionality of the output which is 5.
<br>

## Thanks for reading :)
Created by [Saeed Aghabozorgi](https://www.linkedin.com/in/saeedaghabozorgi/) and modified by [Tarun Kamboj](https://www.linkedin.com/in/kambojtarun/).